In [ ]:
'Ainda sem os comentários necessários -> acrescentar depois + read.me!!'

'Secunda versão Analise Covid -> analisar o arquivo csv do COVID, 
'criando agrupamentos por ano, mes, dia, estado, país e fazer uma análise'

'31/05'
'Versão incompleta!'

In [99]:
from pyspark.sql import SparkSession

In [100]:
spark = SparkSession.builder.appName("covid2").getOrCreate();

In [101]:
dfcovid = spark.read.csv('covid_19_data.csv', inferSchema=True, header=True)

In [102]:
type(dfcovid)

pyspark.sql.dataframe.DataFrame

In [103]:
dfcovid.printSchema()

root
 |-- SNo: integer (nullable = true)
 |-- ObservationDate: string (nullable = true)
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Last Update: string (nullable = true)
 |-- Confirmed: double (nullable = true)
 |-- Deaths: double (nullable = true)
 |-- Recovered: double (nullable = true)



In [104]:
dfcovid.columns

['SNo',
 'ObservationDate',
 'Province/State',
 'Country/Region',
 'Last Update',
 'Confirmed',
 'Deaths',
 'Recovered']

In [105]:
dfcovid.show(5)

+---+---------------+--------------+--------------+---------------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
|  1|     01/22/2020|         Anhui|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  2|     01/22/2020|       Beijing|Mainland China|1/22/2020 17:00|     14.0|   0.0|      0.0|
|  3|     01/22/2020|     Chongqing|Mainland China|1/22/2020 17:00|      6.0|   0.0|      0.0|
|  4|     01/22/2020|        Fujian|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  5|     01/22/2020|         Gansu|Mainland China|1/22/2020 17:00|      0.0|   0.0|      0.0|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
only showing top 5 rows



In [106]:
from pyspark.sql.functions import when,col

In [107]:
from pyspark.sql import SQLContext

In [108]:
from pyspark.sql.functions import format_number

In [109]:
from pyspark.sql.functions import year

In [110]:
from pyspark.sql.functions import unix_timestamp,col, to_date

In [111]:
from pyspark.sql.functions import month

In [112]:
from pyspark.sql.functions import dayofmonth

In [113]:
from pyspark.sql.functions import max, min

In [114]:
from pyspark.sql import SparkSession


In [115]:
from pyspark.sql.functions import *

In [116]:
from pyspark.sql.functions import max, min

In [117]:
from pyspark.sql.functions import col, desc

In [118]:
dfcovid.createOrReplaceTempView("coronaTable")

In [119]:
sql = spark.sql("select * from coronaTable where `Province/State` is null")

In [34]:
dfcovid = dfcovid.withColumn( 'Province/State', when ( col ('Province/State').isNull() , 'N/A' ).otherwise(dfcovid['Province/State'])   )

In [36]:
dfcovid.filter ( dfcovid['Province/State'].isNull() ).show()

+---+---------------+--------------+--------------+-----------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+-----------+---------+------+---------+
+---+---------------+--------------+--------------+-----------+---------+------+---------+



In [38]:
type(sql)

pyspark.sql.dataframe.DataFrame

In [43]:
dfcovid = dfcovid.withColumn("Last Update" , dfcovid["Last Update"] )

In [44]:
dfcovid.show()

+---+---------------+--------------+--------------+---------------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
|  1|     01/22/2020|         Anhui|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  2|     01/22/2020|       Beijing|Mainland China|1/22/2020 17:00|     14.0|   0.0|      0.0|
|  3|     01/22/2020|     Chongqing|Mainland China|1/22/2020 17:00|      6.0|   0.0|      0.0|
|  4|     01/22/2020|        Fujian|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  5|     01/22/2020|         Gansu|Mainland China|1/22/2020 17:00|      0.0|   0.0|      0.0|
|  6|     01/22/2020|     Guangdong|Mainland China|1/22/2020 17:00|     26.0|   0.0|      0.0|
|  7|     01/22/2020|       Guangxi|Mainland China|1/22/2020 17:00|      2.0|   0.0|      0.0|
|  8|     01/22/2020|       Guizhou|Mainland China

In [61]:
#Semana 'corrente' deste ano-> ainda não pega os dados de ObservationDate, está errado!
dfcovid.select(current_date().alias("ObservationDate")
  ).show(1)

+---------------+
|ObservationDate|
+---------------+
|     2021-06-01|
+---------------+
only showing top 1 row



In [63]:
#Separa a tabela ObservationDate conforme o ano,data,dia e semana do ano
dfcovid.select(col("ObservationDate"), 
     year(col("ObservationDate")).alias("year"), 
     month(col("ObservationDate")).alias("month"), 
     next_day(col("ObservationDate"),"Sunday").alias("next_day"), #semana inicia em domingos
     weekofyear(col("ObservationDate")).alias("weekofyear") 
  ).show(60)

+---------------+----+-----+--------+----------+
|ObservationDate|year|month|next_day|weekofyear|
+---------------+----+-----+--------+----------+
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|null| null|    null|      null|
|     01/22/2020|nul

In [78]:
#mudando o formato da data
dfcovid = dfcovid.withColumn('ObservationDate',to_date(unix_timestamp(col("ObservationDate"), 'MM/dd/yyyy').cast('timestamp')))
dfcovid.show(10)

+---+---------------+--------------+--------------+---------------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
|  1|     2020-01-22|         Anhui|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  2|     2020-01-22|       Beijing|Mainland China|1/22/2020 17:00|     14.0|   0.0|      0.0|
|  3|     2020-01-22|     Chongqing|Mainland China|1/22/2020 17:00|      6.0|   0.0|      0.0|
|  4|     2020-01-22|        Fujian|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  5|     2020-01-22|         Gansu|Mainland China|1/22/2020 17:00|      0.0|   0.0|      0.0|
|  6|     2020-01-22|     Guangdong|Mainland China|1/22/2020 17:00|     26.0|   0.0|      0.0|
|  7|     2020-01-22|       Guangxi|Mainland China|1/22/2020 17:00|      2.0|   0.0|      0.0|
|  8|     2020-01-22|       Guizhou|Mainland China

In [89]:
#mostrando apenas o ano da função YEAR
dfYear = dfcovid.select ( year( dfcovid["ObservationDate"] ).alias("DATA") ).show(10)

+----+
|DATA|
+----+
|2020|
|2020|
|2020|
|2020|
|2020|
|2020|
|2020|
|2020|
|2020|
|2020|
+----+
only showing top 10 rows



In [70]:
dfcovid.select( 'SNo',"Province/State", "Country/Region", "Confirmed", 'Deaths', 'Recovered').describe().show()


+-------+----------------+--------------+--------------------+------------------+------------------+-----------------+
|summary|             SNo|Province/State|      Country/Region|         Confirmed|            Deaths|        Recovered|
+-------+----------------+--------------+--------------------+------------------+------------------+-----------------+
|  count|          285307|        285307|              285307|            285307|            285307|           285307|
|   mean|        142654.0|          null|                null| 76629.22584093625|1867.3346605586264|44784.23061824631|
| stddev|82361.1809632831|          null|                null|246617.58933892965|  5905.56555067096|178802.9969423912|
|    min|               1|  Montreal, QC|          Azerbaijan|         -302844.0|            -178.0|        -854405.0|
|    max|          285307|  Zuid-Holland|occupied Palestin...|         5605532.0|          112182.0|        6399531.0|
+-------+----------------+--------------+-------

In [74]:
dfcovidConfZero = dfcovid.filter((dfcovid['Confirmed'] < 0 ) ).show()

+------+---------------+--------------+--------------+-------------------+---------+------+---------+
|   SNo|ObservationDate|Province/State|Country/Region|        Last Update|Confirmed|Deaths|Recovered|
+------+---------------+--------------+--------------+-------------------+---------+------+---------+
|147239|     2020-11-02|       Unknown|      Colombia|2021-04-02 15:13:53|-302844.0|   0.0|      0.0|
+------+---------------+--------------+--------------+-------------------+---------+------+---------+



In [75]:
dfcovidDeathsZero= dfcovid.filter((dfcovid['Deaths'] < 0 ) ).show()

+------+---------------+--------------+--------------+-------------------+---------+------+---------+
|   SNo|ObservationDate|Province/State|Country/Region|        Last Update|Confirmed|Deaths|Recovered|
+------+---------------+--------------+--------------+-------------------+---------+------+---------+
|118117|     2020-09-24|       Unknown|      Colombia|2021-04-02 15:13:53|      0.0|-178.0| -12684.0|
|141257|     2020-10-25|       Unknown|      Colombia|2021-04-02 15:13:53|      0.0|-154.0|  -8072.0|
+------+---------------+--------------+--------------+-------------------+---------+------+---------+



In [77]:
dfcovidReczero = dfcovid.filter((dfcovid['Recovered'] < 0 ) ).show()

+------+---------------+--------------+--------------+-------------------+---------+------+---------+
|   SNo|ObservationDate|Province/State|Country/Region|        Last Update|Confirmed|Deaths|Recovered|
+------+---------------+--------------+--------------+-------------------+---------+------+---------+
|118117|     2020-09-24|       Unknown|      Colombia|2021-04-02 15:13:53|      0.0|-178.0| -12684.0|
|141257|     2020-10-25|       Unknown|      Colombia|2021-04-02 15:13:53|      0.0|-154.0|  -8072.0|
|144995|     2020-10-30|       Unknown|      Colombia|2021-04-02 15:13:53|      0.0| 505.0|-854405.0|
+------+---------------+--------------+--------------+-------------------+---------+------+---------+



In [80]:
#agregação do novo campo
dfGroupByMax = dfcovid.groupBy("Country/Region").max()

In [82]:
dfGroupByMax.sort( col("max(Deaths)").desc() ).show(20)

+--------------+--------+--------------+-----------+--------------+
|Country/Region|max(SNo)|max(Confirmed)|max(Deaths)|max(Recovered)|
+--------------+--------+--------------+-----------+--------------+
|            UK|  285281|     3861901.0|   112182.0|        4186.0|
|        France|  285282|     5605532.0|   104093.0|      307500.0|
|        Brazil|  285221|     2923367.0|    97058.0|     2588973.0|
|          Iran|  284615|     2534855.0|    72484.0|     1988165.0|
|         India|  285285|     4665754.0|    69615.0|     3930302.0|
|        Poland|  284666|     2803233.0|    68068.0|     2520968.0|
|     Argentina|  284550|     3005259.0|    64252.0|     2676197.0|
|            US|  285290|     3744830.0|    62078.0|     6399531.0|
|  South Africa|  284687|     1584064.0|    54417.0|     1507778.0|
|     Indonesia|  284614|     1677274.0|    45796.0|     1530718.0|
|        Turkey|  284703|     4875388.0|    40844.0|     4480381.0|
|        Mexico|  285301|      641997.0|    3400

In [84]:
#agregação do novo campo
dfGroupByMaxState = dfcovid.groupBy("Province/State").max()

In [86]:
dfGroupByMaxState.sort( col("max(Deaths)").desc() ).show(30)

+-----------------+--------+--------------+-----------+--------------+
|   Province/State|max(SNo)|max(Confirmed)|max(Deaths)|max(Recovered)|
+-----------------+--------+--------------+-----------+--------------+
|          England|  284851|     3861901.0|   112182.0|           0.0|
|              N/A|  284714|     5605532.0|   104093.0|     4480381.0|
|        Sao Paulo|  285171|     2923367.0|    97058.0|     2588973.0|
|      Maharashtra|  285006|     4665754.0|    69615.0|     3930302.0|
|       California|  284792|     3744830.0|    62078.0|           6.0|
|         New York|  285064|     2054848.0|    52358.0|           0.0|
|            Texas|  285216|     2897110.0|    50290.0|           0.0|
|   Rio de Janeiro|  285147|      747449.0|    44835.0|      694160.0|
|          Florida|  284858|     2242778.0|    35268.0|           0.0|
|     Minas Gerais|  285029|     1370202.0|    34289.0|     1256330.0|
|           Mexico|  285025|      244629.0|    34008.0|       86405.0|
|     

In [92]:
#agregação do novo campo
dfGroupByYear = dfcovid.groupBy("ObservationDate").max()

In [94]:
dfGroupByYear.show(20)

+---------------+--------+--------------+-----------+--------------+
|ObservationDate|max(SNo)|max(Confirmed)|max(Deaths)|max(Recovered)|
+---------------+--------+--------------+-----------+--------------+
|     2020-08-24|   95078|      756480.0|    36789.0|     2020774.0|
|     2021-01-27|  212727|     3251292.0|    89610.0|     2331314.0|
|     2020-07-24|   71969|      463218.0|    41082.0|     1261624.0|
|     2020-08-05|   80918|      585265.0|    41749.0|     1577851.0|
|     2020-11-29|  167737|     2219502.0|    51977.0|     5065030.0|
|     2020-04-30|   20439|      304372.0|    27967.0|      153947.0|
|     2020-08-28|   98058|      796209.0|    36837.0|     2118367.0|
|     2020-08-29|   98803|      801422.0|    36849.0|     2140614.0|
|     2020-10-04|  125637|     1443409.0|    38084.0|     2911699.0|
|     2021-04-29|  283015|     5547670.0|   112149.0|     4255714.0|
|     2020-03-07|    4038|       67666.0|     2959.0|       43500.0|
|     2020-03-13|    5435|       6

In [95]:
# agregação do novo campo
dfGroupByCount = dfcovid.groupBy("Country/Region").count()

In [96]:
dfGroupByCount.show(20)

+--------------+-----+
|Country/Region|count|
+--------------+-----+
|          Chad|  410|
|        Russia|28010|
|      Paraguay|  421|
| North Ireland|    1|
|         Yemen|  388|
|       Senegal|  427|
|        Sweden| 7098|
|    Cabo Verde|  409|
|        Guyana|  417|
|   Philippines|  460|
|        Jersey|    8|
|       Eritrea|  408|
|         Burma|  402|
|      Djibouti|  411|
|     Singapore|  466|
|      Malaysia|  465|
|          Fiji|  410|
|        Turkey|  418|
|        Malawi|  396|
|          Iraq|  435|
+--------------+-----+
only showing top 20 rows



In [98]:
#mostrando apenas o ano da função MONTH
dfcovid = dfcovid.withColumn ("Month", month( dfcovid["ObservationDate"] ) ).show(100)

+---+---------------+--------------+--------------+---------------+---------+------+---------+-----+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|Month|
+---+---------------+--------------+--------------+---------------+---------+------+---------+-----+
|  1|     2020-01-22|         Anhui|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|    1|
|  2|     2020-01-22|       Beijing|Mainland China|1/22/2020 17:00|     14.0|   0.0|      0.0|    1|
|  3|     2020-01-22|     Chongqing|Mainland China|1/22/2020 17:00|      6.0|   0.0|      0.0|    1|
|  4|     2020-01-22|        Fujian|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|    1|
|  5|     2020-01-22|         Gansu|Mainland China|1/22/2020 17:00|      0.0|   0.0|      0.0|    1|
|  6|     2020-01-22|     Guangdong|Mainland China|1/22/2020 17:00|     26.0|   0.0|      0.0|    1|
|  7|     2020-01-22|       Guangxi|Mainland China|1/22/2020 17:00|      2.0|   0.0|      0